1. Skapa en dataframe med två kolumner. Den första kolumnen innehåller SCB-namn och den andra tillhörande SLOSH-namn
    * använd textfilen från Connies omkodning
  
2. Skapa en dataframe med info från excelfilen från SCB.  
  
3. Slå ihop dessa och droppa de ursprungliga namnen från SCB.
   

# Innehållsförteckning  
1. [Metadata från SCB](#1)  
2. [Skapa itemnr](#2)  
3. [Skapa enkät](#3)  
4. [Kodlistor till db](#4)
5. [SAS-kod som kodar om SCB-variabler](#5)  
6. [Skapa dicts för db](#6)  
7. [????](#7)  
***

In [1]:
import pandas as pd
import numpy as np
import sqlite3

# <a id = "1">1. Metadata från SCB</a>  

Läs in kodboken från SCB.

In [2]:
df2 = pd.read_excel('SCB kodbok.xlsx')
df2.rename(columns = {'Frågenummer':'namn', 'svarsalternativ': 'kod', 'svarsalternativ i klartext': 'svarstext'}, inplace = True)

In [3]:
df2.head()

,namn,fråga i klartext,delfrågor i klartext,kod,svarstext
0,S1,Screeningfråga på webben: Förvärvsarbetar du f...,NaN,1.0,Ja
1,S1,Screeningfråga på webben: Förvärvsarbetar du f...,NaN,2.0,Nej
2,S2,Screeningfråga på webben: Är du ålderspensionä...,NaN,1.0,Ja
3,S2,Screeningfråga på webben: Är du ålderspensionä...,NaN,2.0,Nej
4,S3,Screeningfråga på webben: Har du under de sena...,NaN,1.0,Ja


# <a id = "2">2. Siffrorna i kodlistan</a>  
Gör om siffrorna i kodlistorna till strängar så att decimalen kan tas bort.

In [4]:
df2['kod'] = df2['kod'].astype(str)
df2['kod'] = df2['kod'].str.replace('.0', '')

In [30]:
df2.kod.value_counts()

2      663
1      662
3      586
4      520
5      273
nan    116
6       88
7       31
8        7
9        5
         2
11       1
Name: kod, dtype: int64

# <a id = "2">2. Skapa itemnr</a>  

Det verkar som att itemnr finns i namnkolumnen från SCB. Den första bokstaven är antingen F eller I. F står för "förvärvsarbetande" och I står för "icke-förvärvsarbetande" (misstänker jag). Siffrorna som följer efter F och I verkar vara itemnr.

In [5]:
df2['itemnr'] = df2['namn'].astype(str).str[1:]
df2.head(20)

,namn,fråga i klartext,delfrågor i klartext,kod,svarstext,itemnr
0,S1,Screeningfråga på webben: Förvärvsarbetar du f...,NaN,1,Ja,1
1,S1,Screeningfråga på webben: Förvärvsarbetar du f...,NaN,2,Nej,1
2,S2,Screeningfråga på webben: Är du ålderspensionä...,NaN,1,Ja,2
3,S2,Screeningfråga på webben: Är du ålderspensionä...,NaN,2,Nej,2
4,S3,Screeningfråga på webben: Har du under de sena...,NaN,1,Ja,3
5,S3,Screeningfråga på webben: Har du under de sena...,NaN,2,Nej,3
6,Yrke,a) Vilket är ditt huvudsakliga yrke eller jobb?,NaN,nan,NaN,rke
7,Yrke_1,b) Vilka är dina huvudsakliga arbetsuppgifter?,NaN,nan,Text,rke_1
8,Yrke_2,c) Vilken är den huvudsakliga verksamheten ell...,NaN,nan,Text,rke_2
9,F2,"Är du i ovanstående jobb anställd, lantbrukare...",NaN,1,Anställd av privat företag,2


# <a id = "3">3. Skapa enkät</a>  

F står nog för arbetande och I för icke-arbetande så det borde enkelt gå att ordna en kolumn med enkätvärden.

In [6]:
df2['enkät'] = df2['namn'].astype(str).str[0]

df2.replace({'enkät':{'F': 'SLOSH 20 working', 'I': 'SLOSH 20 non-working'}}, inplace = True)

Kontrollera så att nyckelkolumnerna i de båda tabellerna verkligen är identiska.

# <a id = "4">4. Sammanfoga textkolumnerna</a>  
I filen från SCB finns två textkolumner. Den första är "fråga i klartext" och den andra "delfrågor i klartext". Dessa måste slås samman för att det ska stämma överens med den data som redan finns i relationsdatabasen.

In [7]:
df2.columns

Index(['namn', 'fråga i klartext', 'delfrågor i klartext', 'kod', 'svarstext',
       'itemnr', 'enkät'],
      dtype='object')

In [8]:
df2['enkättext'] = df2['fråga i klartext'] + ' @ ' + df2['delfrågor i klartext'].astype(str)

df2['enkättext'] = df2['enkättext'].str.replace(' @ nan', '')

df2.drop(['fråga i klartext', 'delfrågor i klartext'], axis = 1, inplace = True)

df2 = df2[['namn', 'enkättext', 'kod', 'svarstext', 'itemnr', 'enkät']]

# <a id = "5">5. SAS-kod som kodar om SCB-variabler</a>  

Hämta texten som är kopierad från den syntax Connie använt för att koda om variablerna.

In [9]:
text = open("SCB variabelnamn.txt")

namnstr = text.read().replace("\n", " ")

text.close()

Varje par av namn identifieras av att sista symoblen i SLOSH-namnet alltid är "_8" bryt upp paren och lägg i varsin kolumn i en dataframe.

In [10]:
L1 = [i.strip() + '_8' for i in namnstr.split('_8')]

L2 = [i.split('=') for i in L1]

df1 = pd.DataFrame(L2, columns = ['namn', 'SLOSH'])
df1['namn'] = df1['namn'].str.strip()

Såhär ser tabellen ut:

In [11]:
df1.head(9)

,namn,SLOSH
0,F2,emppfarsel_8
1,F2_txt,empfarseltxt_8
2,F3a,hogutb_8
3,F3b,hogutbar_8
4,F4,antalarb_8
5,F5_01,permanent_8
6,F5_02,project_8
7,F5_03,substitute_8
8,F5_04,hourly_8


In [12]:
for i in range(len(df1.namn)):
    df2.replace(df1.iloc[i, 0], df1.iloc[i, 1], inplace = True)

In [13]:
df2.head(50)

,namn,enkättext,kod,svarstext,itemnr,enkät
0,S1_8,Screeningfråga på webben: Förvärvsarbetar du f...,1,Ja,1,S
1,S1_8,Screeningfråga på webben: Förvärvsarbetar du f...,2,Nej,1,S
2,S2_8,Screeningfråga på webben: Är du ålderspensionä...,1,Ja,2,S
3,S2_8,Screeningfråga på webben: Är du ålderspensionä...,2,Nej,2,S
4,S3_8,Screeningfråga på webben: Har du under de sena...,1,Ja,3,S
5,S3_8,Screeningfråga på webben: Har du under de sena...,2,Nej,3,S
6,yrke_8,a) Vilket är ditt huvudsakliga yrke eller jobb?,nan,NaN,rke,Y
7,yrketxt_8,b) Vilka är dina huvudsakliga arbetsuppgifter?,nan,Text,rke_1,Y
8,yrke3_8,c) Vilken är den huvudsakliga verksamheten ell...,nan,Text,rke_2,Y
9,emppfarsel_8,"Är du i ovanstående jobb anställd, lantbrukare...",1,Anställd av privat företag,2,SLOSH 20 working


# 6.4. Definiera funktionen Nyrad
Funktionen `Nyrad()` tar (namnet på) en tabell, en tuple med kolumnnamn samt en tuple med kolumnvärden. Den skapar en rad av denna information och försöker lägga in den i databasen. Om den lyckas kommer den SQL-kod som används för att skapa raden att noteras i en text-fil. Denna text-fil ska tjäna dels som en backup men även som en logg. Skulle något hända med databasen går det enkelt att bara köra text-filen så återskapas allting.  
  Ifall funktionen inte lyckas lägga in en rad i databasen, kanske p.g.a. att "foregin key" inte respekteras, kommer ett felmeddelande konstatera detta. För mer information om hur SQLite används i python: https://pynative.com/python-sqlite-insert-into-table/.

In [14]:
def Nyrad(tabell, kolumner, värden):    
    try:
        # Koppla databasen
        conn = sqlite3.connect('test.db')

        # Skapa en "cursor"... vet ej vad detta är, men det är nödvändigt.
        cursor = conn.cursor()

        # Denna query ska skrivas i SQL och körs av python mot databasen
        sqlite_insert_query = 'INSERT INTO {} {} VALUES {}'.format(tabell, kolumner, värden)

        # Först måste 'foreign key constraint' slås på, i SQLite är detta avstängt som default. 
        cursor.execute('PRAGMA foreign_keys = 1')

        # Nu kan raden läggas in
        cursor.execute(sqlite_insert_query)

        # commit:a till databasen
        conn.commit()
        print('\033[1m' + 'Rad inlagd i tabellen {}'.format(tabell) + '\033[0m')

        # Stäng
        cursor.close()
        
        print('INSERT INTO {} {} VALUES {}'.format(tabell, kolumner, värden))
        
        # När en rad lagts till ska SQL-kommandot sparas i en textfil.
        text = open("test-logg.txt", "a", encoding = 'UTF-8')
        text.write('INSERT INTO {} {} VALUES {};\n \n'.format(tabell, kolumner, värden))
        text.close()

    except sqlite3.Error as error:
        print('\033[1m' + 'Det gick ej att infoga data: ' + '\033[0m', error)
    finally:
        if (conn):
            conn.close()

# <a id = "6">6. Skapa dicts för db</a>  

Här skär jag ut all information som behövs från df2 för att lägga in metadata med hjälp av avsnitt 2.4 (och framåt) i Arbetsflöde. Informationen för varje item läggs i en dictionary och dessa läggs sedan i en listan som kallas `frågor`.

In [15]:
frågor = []
lista = []
for i in range(len(df2.namn)):
    if i > 0:#undantar den första raden
        if df2.namn[i] != df2.namn[i-1]:#ny kodlista utmärks av ny enkättext
            
            variabel = df2.namn[i-1]
            
            enkät = df2.enkät[i-1]
            
            kodsvar = {kod:svarstext for (kod, svarstext) in lista}
            
            if lista == []: #fr variabler med kodlistor med bara en rad
                kodsvar = {df2.kod[i]:df2.svarstext[i]}
            
            itemnr = df2.itemnr[i-1]

            enkättext = df2.enkättext[i-1]

            dubbelkodning = ''
            
            beskrivning = ''
            
            enfråga = {'var':variabel, 'enk':enkät, 'kods':kodsvar, 'item':itemnr, 'enkä':enkättext, 'dubb':dubbelkodning, 'besk':beskrivning}
            
            frågor.append(enfråga)
            
            lista = []
            
    par = (df2.iloc[i, 2], df2.iloc[i, 3])
    lista.append(par)

In [16]:
frågor

[{'var': 'S1_8',
  'enk': 'S',
  'kods': {'1': 'Ja', '2': 'Nej'},
  'item': '1',
  'enkä': 'Screeningfråga på webben: Förvärvsarbetar du för närvarande halvtid eller mer?',
  'dubb': '',
  'besk': ''},
 {'var': 'S2_8',
  'enk': 'S',
  'kods': {'1': 'Ja', '2': 'Nej'},
  'item': '2',
  'enkä': 'Screeningfråga på webben: Är du ålderspensionär, helt förtidspensionerad, heltidsarbetslös, studerande eller hemarbetande på heltid?',
  'dubb': '',
  'besk': ''},
 {'var': 'S3_8',
  'enk': 'S',
  'kods': {'1': 'Ja', '2': 'Nej'},
  'item': '3',
  'enkä': 'Screeningfråga på webben: Har du under de senaste 3 månaderna i genomsnitt förvärvsarbetat (gått till arbetet) 30 % eller mer?',
  'dubb': '',
  'besk': ''},
 {'var': ' yrke_8',
  'enk': 'Y',
  'kods': {'nan': nan},
  'item': 'rke',
  'enkä': 'a) Vilket är ditt huvudsakliga yrke eller jobb?',
  'dubb': '',
  'besk': ''},
 {'var': 'yrketxt_8',
  'enk': 'Y',
  'kods': {'nan': 'Text'},
  'item': 'rke_1',
  'enkä': 'b) Vilka är dina huvudsakliga arbe

* Fråga Connie vad hon vill att jag ska ersätta S, Y E och T med.  
* Hur ska jag gör amed missing och slask? Finns de i senaste vågen?  
* vad gör jag med dubbelkodning?

In [18]:
frågor[0]

{'var': 'S1_8',
 'enk': 'S',
 'kods': {'1': 'Ja', '2': 'Nej'},
 'item': '1',
 'enkä': 'Screeningfråga på webben: Förvärvsarbetar du för närvarande halvtid eller mer?',
 'dubb': '',
 'besk': ''}

# 7. Infoga rader i databasen  
Nu kommer den transkriberade informationen att användas tillsamman med funktionen `Nyrad()` för var och en av tabellerna i databasen.

In [16]:
for i in range(len(frågor)):
    variabel = frågor[i]['var'] 
    
    enkät = frågor[i]['enk']

    beskrivning = frågor[i]['besk']

    itemnr = frågor[i]['item']

    enkättext = frågor[i]['enkä']

    dubbelkodning = ''

    kodsvar = frågor[i]['kods']

    # 6.2 Finns kodlistan redan? För att ta ställning till ifall kodlistan redan förekommer i databasen hämtas först all information i *Kodlistor_stor*. Ifall kodlistan redan finns sätts variabeln `kodlista` lika med det nummer kodlistan redan har i databasen. Om den *inte* finns sätts `kodlista` $ = $ 'NULL'. Ifall SQLite får detta som värde, och kolumnen ifråga är s.k. "AUTOINCREMENT", kommer kodlistan automatiskt få ett nytt heltal som id.
    # Koppla databasen
    conn = sqlite3.connect('test.db')

    # Skapa en "cursor"... vet ej vad detta är, men det är nödvändigt.
    cursor = conn.cursor()

    # Denna query ska skrivas i SQL och körs av python mot databasen.
    cursor.execute('SELECT * FROM Kodlistor_stor')

    # Detta är en lista med vektorer. Varje vektor representerar en rad i tabellen.
    rows = cursor.fetchall()

    # Stäng databasen.
    cursor.close()

    kods = []
    L = []

    # gå igenom varje vektor i listan
    for i in range(len(rows)):

        # Gäller bara första raden. Detta görs för att undvika index-fel nedan (i-1)
        if len(L) == 0:

            # Välj ut kod och svarstext.
            a = (rows[i][1], rows[i][2])

            # Lägg i a.
            L.append(a)
        else:

            # Ifall k_id skiljer från nuvarnade har loopen nått en ny kodlista. Skapa en dict av innehållet i L och
            # nollställ sedan L inför nästa kodlista.
            if rows[i][0] != rows[i-1][0]:
                d = {j[0]: j[1] for j in L}
                k = rows[i-1][0]
                D = [k, d]
                kods.append(D)
                L = []

            if i == (len(rows) - 1):
                a = (rows[i][1], rows[i][2])
                L.append(a)
                d = {j[0]: j[1] for j in L}
                k = rows[i][0]
                D = [k, d]
                kods.append(D)
                break



            a = (rows[i][1], rows[i][2])
            L.append(a)


    nykodlista = 'ja'
    kodlista = 'NULL'

    # Om kodlistan redan finns hämtas dess namn och läggs i variabeln kodlista.
    for i in kods:
        if kodsvar == i[1]:
            kodlista = i[0]
            nykodlista = 'nej'
            break

    # 6.3 Finns svarstexterna redan? För att slippa att loopen som lägger in svarstexter nedan avbryts halvvägs igenom av att någon svarstext redan finns i databasen filtreras svarstexterna här. Om de redan är inlagda görs inget försök att lägga in dem igen.
    # Koppla databasen
    conn = sqlite3.connect('test.db')

    # Skapa en "cursor"... vet ej vad detta är, men det är nödvändigt.
    cursor = conn.cursor()

    # Denna query ska skrivas i SQL och körs av python mot databasen.
    cursor.execute('SELECT * FROM Svarstexter')

    # Detta är en lista med vektorer. Varje vektor representerar en rad i tabellen.
    rows = cursor.fetchall()

    # Stäng databasen.
    cursor.close()

    # De transkriberade svarstexterna.
    svarstexter = list(kodsvar.values())

    # rows innehåller vektorer. Här plockat texten ut från vektorerna.
    dbsvar = [i[0] for i in rows]

    # En lista med de svarstexter som inte redan finns i databasen.
    filtsvar = [i for i in svarstexter if i not in dbsvar]

    if len(filtsvar) > 0:
        print('\033[1m' + 'Nya svarstexter:' + '\033[0m', filtsvar)
        filtsvar
    else:
        print('\033[1m' + 'Inga nya svarstexter' + '\033[0m')

    # 7. Infoga rader i databasen

    # C: Kodlistor
    tabell = 'Kodlistor'

    kolumner = '(kodlista)'

    värden = '({})'.format(kodlista)

    if kodlista == 'NULL':
        Nyrad(tabell, kolumner, värden)

    # G: Enkättexter    
    tabell = 'Enkättexter'

    kolumner = '(enkättext)'

    värden = '(\'{}\')'.format(enkättext)

    Nyrad(tabell, kolumner, värden)

    # H: Svarstexter
    tabell = 'Svarstexter'

    kolumner = '(svarstext)'

    if len(filtsvar) > 0:
        for i in filtsvar:
            värden = '(\'{}\')'.format(i)
            Nyrad(tabell, kolumner, värden)

    # D: Kodlistor_stor
    tabell = 'Kodlistor_stor'

    kolumner = ('KODLISTA', 'KOD', 'SVARSTEXT')

    if kodlista == 'NULL':

        # Koppla databasen
        conn = sqlite3.connect('test.db')

        # Skapa en "cursor"... vet ej vad detta är, men det är nödvändigt.
        cursor = conn.cursor()

        # Denna query ska skrivas i SQL och körs av python mot databasen.
        cursor.execute('SELECT kodlista FROM Kodlistor')

        # Detta är en lista med vektorer. Varje vektor representerar en rad i tabellen.
        rows = cursor.fetchall()

        # Stäng databasen.
        cursor.close()

        kodlista = str(rows[-1][0])

    if nykodlista == 'ja':
        for i in kodsvar.keys():
            värden = '(\'{}\', \'{}\', \'{}\')'.format(kodlista, i, kodsvar[i])
            #värden = ('NULL', kodlista, i, kodsvar[i])
            Nyrad(tabell, kolumner, värden)

    # B: Variabler
    tabell = 'Variabler'

    kolumner = ('VARIABEL', 'enkät', 'BESKRIVNING', 'DUBBELKODNING', 'ITEMNR', 'enkättext', 'KODLISTA')

    # värden = '(variabel, enkät, beskrivning, dubbelkodning, itemnr, enkättext, kodlista)

    värden = '(\'{}\', \'{}\', \'{}\', \'{}\', \'{}\', \'{}\', \'{}\')'.format(variabel, enkät, beskrivning, 
                                                                               dubbelkodning, itemnr, enkättext,
                                                                              kodlista)
    Nyrad(tabell, kolumner, värden)

Nya svarstexter: ['Ja', 'Nej']
Rad inlagd i tabellen Kodlistor
INSERT INTO Kodlistor (kodlista) VALUES (NULL)
Rad inlagd i tabellen Enkättexter
INSERT INTO Enkättexter (enkättext) VALUES ('Screeningfråga på webben: Förvärvsarbetar du för närvarande halvtid eller mer?')
Rad inlagd i tabellen Svarstexter
INSERT INTO Svarstexter (svarstext) VALUES ('Ja')
Rad inlagd i tabellen Svarstexter
INSERT INTO Svarstexter (svarstext) VALUES ('Nej')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('781', '1', 'Ja')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('781', '2', 'Nej')
Rad inlagd i tabellen Variabler
INSERT INTO Variabler ('VARIABEL', 'enkät', 'BESKRIVNING', 'DUBBELKODNING', 'ITEMNR', 'enkättext', 'KODLISTA') VALUES ('S1_8', 'S', '', '', '1', 'Screeningfråga på webben: Förvärvsarbetar du för närvarande halvtid eller mer?', '781')
Inga nya svarstexter
Rad inlagd i tabellen Kodli

Rad inlagd i tabellen Svarstexter
INSERT INTO Svarstexter (svarstext) VALUES ('0')
Rad inlagd i tabellen Svarstexter
INSERT INTO Svarstexter (svarstext) VALUES ('1 till 9')
Rad inlagd i tabellen Svarstexter
INSERT INTO Svarstexter (svarstext) VALUES ('10 eller fler')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('791', '1', '0')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('791', '2', '1 till 9')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('791', '3', '10 eller fler')
Rad inlagd i tabellen Variabler
INSERT INTO Variabler ('VARIABEL', 'enkät', 'BESKRIVNING', 'DUBBELKODNING', 'ITEMNR', 'enkättext', 'KODLISTA') VALUES (' antalarb_8', 'SLOSH 20 working', '', '', '4', 'Hur många anställda finns på din arbetsplats, utöver dig själv?', '791')
Nya svarstexter: ['Tillsvidareanställning/Fast anställning']
Rad inlagd i 

Rad inlagd i tabellen Svarstexter
INSERT INTO Svarstexter (svarstext) VALUES ('personer')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('802', 'nan', 'personer')
Rad inlagd i tabellen Variabler
INSERT INTO Variabler ('VARIABEL', 'enkät', 'BESKRIVNING', 'DUBBELKODNING', 'ITEMNR', 'enkättext', 'KODLISTA') VALUES (' buissize_8', 'SLOSH 20 working', '', '', '6c', 'c) Hur många medarbetare har företaget?', '802')
Nya svarstexter: ['Uppdragen fördelar sig jämt över året', 'Uppdragsläget varierar starkt eller är säsongsberoende']
Rad inlagd i tabellen Kodlistor
INSERT INTO Kodlistor (kodlista) VALUES (NULL)
Rad inlagd i tabellen Enkättexter
INSERT INTO Enkättexter (enkättext) VALUES ('d) Hur fördelar sig dina uppdrag över året?')
Rad inlagd i tabellen Svarstexter
INSERT INTO Svarstexter (svarstext) VALUES ('Uppdragen fördelar sig jämt över året')
Rad inlagd i tabellen Svarstexter
INSERT INTO Svarstexter (svarstext) VALUES ('Uppdragslä

INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('808', '1', 'Aldrig eller nästa aldrig')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('808', '2', 'Ganska sällan')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('808', '3', 'Ungefär hälften av arbetstiden')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('808', '4', 'Ganska ofta')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('808', '5', 'Nästan alltid eller alltid')
Rad inlagd i tabellen Variabler
INSERT INTO Variabler ('VARIABEL', 'enkät', 'BESKRIVNING', 'DUBBELKODNING', 'ITEMNR', 'enkättext', 'KODLISTA') VALUES (' selffam_8', 'SLOSH 20 working', '', '', '6g_03', 'g) Hur ofta arbetar du som egenföretagare… @ …tillsammans med familjemedlemmar(som inte är anställda i företag)?', '808')
In

Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('815', '2', '1-4 personer')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('815', '3', '5-9 personer')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('815', '4', '10-19 personer')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('815', '5', '20 personer eller fler')
Rad inlagd i tabellen Variabler
INSERT INTO Variabler ('VARIABEL', 'enkät', 'BESKRIVNING', 'DUBBELKODNING', 'ITEMNR', 'enkättext', 'KODLISTA') VALUES (' teamsize_8', 'SLOSH 20 working', '', '', '10', 'Hur stor är den arbetsgrupp du arbetar i?', '815')
Nya svarstexter: ['Stämmer helt och hållet', 'Stämmer ganska bra', 'Stämmer inte särskilt bra', 'Stämmer inte alls', 'Vet inte/inte aktuellt']
Rad inlagd i tabellen Kodlistor
INSERT INTO Kodlistor (kodlis

...

Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('1521', '4', 'Ganska mycket')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('1521', '5', 'Väldigt mycket')
Rad inlagd i tabellen Variabler
INSERT INTO Variabler ('VARIABEL', 'enkät', 'BESKRIVNING', 'DUBBELKODNING', 'ITEMNR', 'enkättext', 'KODLISTA') VALUES ('climstress_8', 'SLOSH 20 non-working', '', '', '83a', '83. Om den globala uppvärmningen @ Hur mycket oroar du dig för klimatförändringarna?', '1521')
Inga nya svarstexter
Rad inlagd i tabellen Kodlistor
INSERT INTO Kodlistor (kodlista) VALUES (NULL)
Rad inlagd i tabellen Enkättexter
INSERT INTO Enkättexter (enkättext) VALUES ('83. Om den globala uppvärmningen @ Hur mycket anpassar du din livsstil för att minska ditt klimatavtryck (till exempel äter vegetariskt/veganskt, reser fossilfritt, konsumerar miljömedvetet)?')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodli

Inga nya svarstexter
Rad inlagd i tabellen Kodlistor
INSERT INTO Kodlistor (kodlista) VALUES (NULL)
Rad inlagd i tabellen Enkättexter
INSERT INTO Enkättexter (enkättext) VALUES ('87. Lite om hur du ser på ditt liv… @ Jag tycker att framtiden ser bra ut för min del.')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('1531', '1', 'Ofta')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('1531', '2', 'Ibland')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('1531', '3', 'Sällan')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('1531', '4', 'Aldrig')
Rad inlagd i tabellen Variabler
INSERT INTO Variabler ('VARIABEL', 'enkät', 'BESKRIVNING', 'DUBBELKODNING', 'ITEMNR', 'enkättext', 'KODLISTA') VALUES ('futugood_8', 'SLOSH 20 non-working', '', '', '87a', '87. Lite om hur d

Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('1539', '3', 'Sällan')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('1539', '4', 'Aldrig')
Rad inlagd i tabellen Variabler
INSERT INTO Variabler ('VARIABEL', 'enkät', 'BESKRIVNING', 'DUBBELKODNING', 'ITEMNR', 'enkättext', 'KODLISTA') VALUES ('lifemean_8', 'SLOSH 20 non-working', '', '', '87i', '87. Lite om hur du ser på ditt liv… @ Jag känner att mitt liv har mening.', '1539')
Inga nya svarstexter
Rad inlagd i tabellen Kodlistor
INSERT INTO Kodlistor (kodlista) VALUES (NULL)
Rad inlagd i tabellen Enkättexter
INSERT INTO Enkättexter (enkättext) VALUES ('87. Lite om hur du ser på ditt liv… @ Jag uppskattar de saker jag gör.')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('1540', '1', 'Ofta')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 

Inga nya svarstexter
Rad inlagd i tabellen Kodlistor
INSERT INTO Kodlistor (kodlista) VALUES (NULL)
Rad inlagd i tabellen Enkättexter
INSERT INTO Enkättexter (enkättext) VALUES ('87. Lite om hur du ser på ditt liv… @ Jag ser på det hela taget tillbaka på mitt liv med  en känsla av lycka.')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('1547', '1', 'Ofta')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('1547', '2', 'Ibland')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('1547', '3', 'Sällan')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('1547', '4', 'Aldrig')
Rad inlagd i tabellen Variabler
INSERT INTO Variabler ('VARIABEL', 'enkät', 'BESKRIVNING', 'DUBBELKODNING', 'ITEMNR', 'enkättext', 'KODLISTA') VALUES ('hppiness_8', 'SLOSH 20 non-working', '', '', '8

Inga nya svarstexter
Rad inlagd i tabellen Kodlistor
INSERT INTO Kodlistor (kodlista) VALUES (NULL)
Det gick ej att infoga data:  UNIQUE constraint failed: Enkättexter.enkättext
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('1556', 'nan', 'Text')
Rad inlagd i tabellen Variabler
INSERT INTO Variabler ('VARIABEL', 'enkät', 'BESKRIVNING', 'DUBBELKODNING', 'ITEMNR', 'enkättext', 'KODLISTA') VALUES ('webcomment_8', 'SLOSH 20 non-working', '', '', '94_txt', 'Avslutande frågor på webben: 4. Beskriv gärna vad du tyckte var positivt / negativt med hur själva webblösningen ser ut och fungerar:', '1556')
Inga nya svarstexter
Rad inlagd i tabellen Kodlistor
INSERT INTO Kodlistor (kodlista) VALUES (NULL)
Det gick ej att infoga data:  UNIQUE constraint failed: Enkättexter.enkättext
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('1557', 'nan', 'Dator')
Rad inlagd i tabellen Variabler
I